# Applying metrics and cross-validation

## 1. Setting up your environment: Assume a virtual env was created and all dep. in the requierements.txt file installed

## 2. Importing required libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, r2_score

## 3. Loading and preparing the data

In [2]:
# Sample dataset: Study hours, previous exam scores, and pass/fail labels
data = {
    'StudyHours': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'PrevExamScore': [30, 40, 45, 50, 60, 65, 70, 75, 80, 85],
    'Pass': [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]  # 0 = Fail, 1 = Pass
}

df = pd.DataFrame(data)

# Features and target variable
X = df[['StudyHours', 'PrevExamScore']]
y = df['Pass']

## 4. Applying evaluation metrics without cross-validation

In [3]:
from sklearn.linear_model import LogisticRegression

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Logistic Regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

**Next, calculate the model’s accuracy, precision, recall, and F1 score using the test set predictions:**

In [4]:
# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0


**While the above method works, it’s limited by the single train-test split, which could lead to overfitting or underfitting. To get a more reliable performance estimate, we should use cross-validation. (cross-validation allows you to split the dataset into multiple subsets and reliably calculate model performance).**

## 5. Performing k-fold cross-validation

**Here we use k-fold cross-validation, where the dataset is split into k equal parts (folds). Each fold is used as a test set while the remaining folds are used for training:**

In [5]:
from sklearn.model_selection import cross_val_score

# Initialize the model
model = LogisticRegression()

# Perform 5-fold cross-validation and calculate accuracy for each fold
cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

# Display the accuracy for each fold and the mean accuracy
print(f'Cross-validation accuracies: {cv_scores}')
print(f'Mean cross-validation accuracy: {np.mean(cv_scores)}')

Cross-validation accuracies: [1.  1.  1.  1.  0.5]
Mean cross-validation accuracy: 0.9


**Here, the cross_val_score function automatically splits the data into five folds, trains the model on four folds, and tests it on the remaining fold. This process is repeated five times, and it reports the accuracy for each fold.**

## 6. Cross-validation with multiple metrics

In [6]:
from sklearn.model_selection import cross_validate

# Define multiple scoring metrics
scoring = ['accuracy', 'precision', 'recall', 'f1']

# Perform cross-validation
cv_results = cross_validate(model, X, y, cv=5, scoring=scoring)

# Print results for each metric
print(f"Cross-validation Accuracy: {np.mean(cv_results['test_accuracy'])}")
print(f"Cross-validation Precision: {np.mean(cv_results['test_precision'])}")
print(f"Cross-validation Recall: {np.mean(cv_results['test_recall'])}")
print(f"Cross-validation F1-Score: {np.mean(cv_results['test_f1'])}")

Cross-validation Accuracy: 0.9
Cross-validation Precision: 0.9
Cross-validation Recall: 1.0
Cross-validation F1-Score: 0.9333333333333333


**This approach presents the average metrics for each fold, providing a better picture of how the model performs across different data splits.**

## 7. Cross-validation with a regression model

**For regression tasks, use metrics such as mean absolute error (MAE), mean squared error (MSE), and R-squared. Apply these metrics with cross-validation for a regression model:**

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Sample dataset for regression
X_reg = df[['StudyHours']]
y_reg = df['PrevExamScore']

# Initialize a linear regression model
reg_model = LinearRegression()

# Perform 5-fold cross-validation using R-squared as the metric
cv_scores_r2 = cross_val_score(reg_model, X_reg, y_reg, cv=5, scoring='r2')

print(f'Cross-validation R-squared scores: {cv_scores_r2}')
print(f'Mean R-squared score: {np.mean(cv_scores_r2)}')

Cross-validation R-squared scores: [ 0.52933673  0.88503086 -0.60298929  0.88503086 -1.28939909]
Mean R-squared score: 0.08140201560607467


**For regression models:**

-- R-squared indicates how well the model explains the variance in the target variable.

-- MSE and MAE measure the average error between the predicted and actual values.